# Sample Code for Google Colab

## Setting
install modules  
import modules  
define function and paths.  

In [1]:
%%bash
# clone our repository.
git clone https://github.com/ou-medinfo/medbertjp.git

# install transformers
pip install transformers

# install a Japanese tokenizer and morphological analysis tools.
# apt-get install mecab    # included in fugashi
pip install fugashi ipadic unidic
python -m unidic download

  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5c1350ad368534e1a824e36003d2c6e353cf16cdd93b235ce2d370c136ed4bc1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for ipadic: filename=ipadic-1.0.0-cp36-none-any.whl size=13556725 sha256=33d430dc881d0c710f01b866613c2494cbcc5d7bd88ac1b5101798a6609b40de
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
  Created wheel for unidic: filename=unidic-1.0.3-cp36-none-any.whl size=5498 sha256=a79c357a99abe1628856a21dde79bb02c763967bc3a4534e52a1cf9a455d3849
  Stored in directory: /root/.cache/pip/wheels/d3/26/e2/fb76c79fd14391eb994eab021c9129c24814125298e1e5b96a
Successfully built ipadic unidic
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic.zip
Dictionary version: 2.3.0+2020-10-08
Finished download.


Cloning into 'medbertjp'...
unidic.zip: 100%|██████████| 608M/608M [00:33<00:00, 17.9MB/s]
Downloaded UniDic v2.3.0+2020-10-08 to /usr/local/lib/python3.6/dist-packages/unidic/dicdir


In [2]:
import random
import torch
from medbertjp.tokenization_jp_mod import BertJapaneseTokenizer
from transformers.modeling_bert import BertForMaskedLM

In [3]:
def prediction_masked_sentence(sentence, tokenizer, model):
    mask_token_id = tokenizer.mask_token_id
    
    input_ids = tokenizer.encode(text, return_tensors='pt')
    print(' '.join(tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])))
    masked_index = random.randint(1, len(input_ids[0]) - 2)
    input_ids[0][masked_index] = mask_token_id
    print(' '.join(tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])))
    
    result = model(input_ids)
    topk = torch.nn.Softmax(dim=1)(result[0][:, masked_index]).topk(5)
    scores = topk.values.tolist()[0]
    pred_ids = topk.indices.tolist()[0]
    for i, pred_id in enumerate(pred_ids):
        output_ids = input_ids.tolist()[0]
        output_ids[masked_index] = pred_id
        
        pred_sentence = tokenizer.convert_ids_to_tokens(output_ids)
        pred_sentence = [token if n != masked_index else '【' + token + '】'
                         for n, token in enumerate(pred_sentence)]
        print('{:.1f} %'.format(scores[i] * 100).rjust(7), ' '.join(pred_sentence))
        
    return result

In [4]:
text = '慢性膵炎では，糖尿病性ケトアシドーシス(DKA)の合併は比較的稀である．'

# Model path
PathToBertJpMecabUnidic = './medBERTjp_L12_H768_A12_WWM_mecab-unidic'
PathToBertJpMecabIpadic = './medBERTjp_L12_H768_A12_WWM_mecab-ipadic'
PathToBertJpSentencepiece = './medBERTjp_L12_H768_A12_sentencepiece'

##  MeCab-IPAdic-tokenized Japanese BERT model
For general sentences.

In [5]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking',
                                                  word_tokenizer_type='mecab',
                                                  mecab_kwargs={
                                                     "mecab_dic": 'ipadic',
                                                 })
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [87]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
[CLS] 慢性 [UNK] で は , 糖尿 [MASK] 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 97.4 % [CLS] 慢性 [UNK] で は , 糖尿 【病】 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.7 % [CLS] 慢性 [UNK] で は , 糖尿 【様】 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.5 % [CLS] 慢性 [UNK] で は , 糖尿 【酸】 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.3 % [CLS] 慢性 [UNK] で は , 糖尿 【質】 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.2 % [CLS] 慢性 [UNK] で は , 糖尿 【床】 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]


## medBERTjp - SentencePiece
For medical sentences.

In [ ]:
%%bash
wget https://github.com/ou-medinfo/medbertjp/releases/download/v0.1-sp/medBERTjp_L12_H768_A12_sentencepiece.zip
unzip ./medBERTjp_L12_H768_A12_sentencepiece.zip

In [89]:
model_path = PathToBertJpSentencepiece
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path,
                                                  word_tokenizer_type='sp',
                                                 )
model = BertForMaskedLM.from_pretrained(model_path)

medBERTjp_L12_H768_A12_sentencepiece/vocab.model
Loaded a trained SentencePiece model.


Some weights of the model checkpoint at ./medBERTjp_L12_H768_A12_sentencepiece were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [121]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
[CLS] ▁ 慢性膵炎 では , [MASK] ( D KA ) の合併 は比較的 稀である . [SEP]
 19.1 % [CLS] ▁ 慢性膵炎 では , 【糖尿病】 ( D KA ) の合併 は比較的 稀である . [SEP]
 18.9 % [CLS] ▁ 慢性膵炎 では , 【急性膵炎】 ( D KA ) の合併 は比較的 稀である . [SEP]
 12.0 % [CLS] ▁ 慢性膵炎 では , 【糖尿病性ケトアシドーシス】 ( D KA ) の合併 は比較的 稀である . [SEP]
  7.4 % [CLS] ▁ 慢性膵炎 では , 【膵癌】 ( D KA ) の合併 は比較的 稀である . [SEP]
  2.7 % [CLS] ▁ 慢性膵炎 では , 【Sjögren】 ( D KA ) の合併 は比較的 稀である . [SEP]


## medBERTjp - MeCab with IPAdic
For medical sentences.

In [ ]:
%%bash
wget https://github.com/ou-medinfo/medbertjp/releases/download/v0.1-mi/medBERTjp_L12_H768_A12_WWM_mecab-ipadic.zip
unzip ./medBERTjp_L12_H768_A12_WWM_mecab-ipadic.zip

In [123]:
model_path = PathToBertJpMecabIpadic
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path, 
                                                 mecab_kwargs={
                                                     "mecab_dic": 'ipadic',
                                                 })
model = BertForMaskedLM.from_pretrained(model_path)

Some weights of the model checkpoint at ./medBERTjp_L12_H768_A12_WWM_mecab-ipadic were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [124]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] 慢性 膵炎 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
[CLS] 慢性 膵炎 で は , 糖尿 [MASK] 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
100.0 % [CLS] 慢性 膵炎 で は , 糖尿 【病】 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.0 % [CLS] 慢性 膵炎 で は , 糖尿 【アルコール】 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.0 % [CLS] 慢性 膵炎 で は , 糖尿 【原】 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.0 % [CLS] 慢性 膵炎 で は , 糖尿 【質】 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.0 % [CLS] 慢性 膵炎 で は , 糖尿 【脈】 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]


## medBERTjp - MeCab with Unidic
is not available now, but will be uploaded soon.


In [ ]:
# model_path = PathToBertJpMecabUnidic
# tokenizer = BertJapaneseTokenizer.from_pretrained(model_path, 
#                                                  mecab_kwargs={
#                                                      "mecab_dic": 'unidic',
#                                                  })
# model = BertForMaskedLM.from_pretrained(model_path)

In [ ]:
# # An Example of Masked Language Modeling.
# result = prediction_masked_sentence(text, tokenizer, model)